# Prototype Development

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from utils import *
from plots import *

data_path = Path("../data/Pedestrian_Counting_System___2009_to_Present__counts_per_hour_.csv")

First we'll prep some dataset metadata that we'll need: 

In [2]:
foot_df = load_and_clean_pedestrian_data(data_path)

In [3]:
import calendar

MONTHS = list(calendar.month_name)[1:]
WEEKDAYS = list(calendar.day_name)
YEARS = sorted(foot_df["Year"].unique())
SENSORS = sorted(foot_df["Sensor_Name"].unique())
NUM_SNESORS = len(SENSORS)

## Ipywidgets

* input controls
* display outputs
* binding inputs and outputs to interactions

In [4]:
import ipywidgets as widgets

In [5]:
widgets.Dropdown(options=SENSORS)

Dropdown(options=('231 Bourke St', 'Alfred Place', 'Australia on Collins', 'Birrarung Marr', 'Bourke St - Spen…

In [6]:
widgets.Dropdown(options=YEARS)

Dropdown(options=(2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020), value=2009)

In [7]:
first_year = min(YEARS)
last_year = max(YEARS)
widgets.IntRangeSlider(
    value=[first_year, last_year],
    min=first_year,
    max=last_year,
)

IntRangeSlider(value=(2009, 2020), max=2020, min=2009)

## Mockup

In [52]:
# inputs

year_input = widgets.Dropdown(options=YEARS, value=YEARS[-1])
sensor_input = widgets.SelectMultiple(options=SENSORS)
month_input = widgets.Dropdown(options=MONTHS)


# outputs

month_counts_output = widgets.interactive_output(
    display_output(functools.partial(plot_month_counts, foot_df)),
    {"sensor": sensor_input, "year": year_input},
)

sensor_counts_output = widgets.interactive_output(
    display_output(functools.partial(plot_sensor_counts, foot_df)),
    {"sensor": sensor_input, "year": year_input},
)

sensors_output = widgets.interactive_output(
    display_output(functools.partial(plot_sensors, foot_df)),
    {"sensor": sensor_input, "year": year_input, "month": month_input},
)


# layout

row1 = widgets.HBox(
    [
        widgets.VBox(
            [
                widgets.HTML("<H1>Melbourne City Council Pedestrian Traffic</h1>"),
                year_input,
                sensor_input,
                month_input,
            ]
        ),
        widgets.VBox(),  # map goes here
    ]
)

row2 = widgets.HBox(
    [
        month_counts_output,
        widgets.HBox([sensor_counts_output], layout={"height": "400px"}),
    ]
)

row3 = widgets.HBox([sensors_output])

widgets.VBox([row1, row2, row3])

# TODO:
# -- month filter needs to be moved to below top two charts; probably shouldn't apply to top sensor list
# -- fix bug in sensor plot list when all are selected
# -- add plot titles
# -- add labels to the dropdowns
# -- make a map plot

## Comparing Sensors by Year

In [19]:
import functools

# inputs
sensor_input = widgets.Dropdown(options=SENSORS)
year_input = widgets.Dropdown(options=YEARS)
month_input = widgets.Dropdown(options=MONTHS)

# binding inputs to output function
out = widgets.interactive_output(
    display_output(functools.partial(plot_years, foot_df)),
    {"sensor": sensor_input, "month": month_input},
)

# layout
widgets.HBox(
    [
        widgets.VBox(
            [
                widgets.HTML("<H1>Comparing Sensors by Year</h1>"),
                sensor_input,
                month_input,
            ]
        ),
        out,
    ]
)

Ideas:
* align to eg first Monday of the month
* get rid of the 2000 from the initial x value see `tickformatstops` example near bottom of [this page](https://plotly.com/python/time-series/)
* extend to `sensor_plot`
* add ability to overlay mean/median lines from custom date (ranges)
* work out how to combine `plot_sensors` and `plot_year` into single `plot_comparison` that also allows selecting month, and even day?
* improve hover label